In [1]:
import pandas as pd

In [2]:
#link do folderu, w którym mamy bazę:

%cd "C:\Users\Dawid\Desktop\kurs\projekt\uber"

C:\Users\Dawid\Desktop\kurs\projekt\uber


In [3]:
df=pd.read_csv("./database.csv")

C:\Users\Dawid\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df.drop(columns=["Unnamed: 0","HOSPSTAY",'DOF1', 'GAD1','DVEST', 
                 'DVBASIS','BAGDEPLY','BAGFAIL','CASENO','MANAVAIL', 'MANFAIL', 'TREATMNT','DEATH'
                ],inplace = True)




Aby sprawnie pracować na danych zawartych w tabeli, podzielimy ją na 3 tabele połacząne kluczem głównym (CASEID i PSU)



In [5]:
occupants = df[["CASEID", 'PSU','YEAR','AGE', 'BAGAVAIL','CHTYPE', 'HEIGHT', 'INJSEV',
       'MANUSE', 'OCCNO', 'ROLE', 'SEATPOS', 'SEX', 'WEIGHT']]

In [6]:
occupants.drop_duplicates(subset=["CASEID", 'PSU','YEAR','AGE', 'BAGAVAIL','CHTYPE', 'HEIGHT', 'INJSEV',
       'MANUSE', 'OCCNO', 'ROLE', 'SEATPOS', 'SEX', 'WEIGHT'],inplace=True, ignore_index=True)

C:\Users\Dawid\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [7]:
vehicles= df[['CASEID', 'PSU','YEAR','BODYTYPE', 'MAKE' , 'DVTOTAL','MODELYR']]

In [8]:
vehicles.drop_duplicates(subset=['CASEID', 'PSU','YEAR','BODYTYPE', 'MAKE' , 'DVTOTAL','MODELYR'],inplace=True, ignore_index=True)

In [9]:
accidents = df[['CASEID', 'PSU', 'YEAR','STRATIF']]

In [10]:
accidents.drop_duplicates(subset=['CASEID', 'PSU', 'YEAR','STRATIF'], inplace =True, ignore_index=True)

In [11]:
import math
x = float('nan')
math.isnan(x)

True

KOLUMNA HEIGHT

In [12]:
Height_new=[]
for i in occupants['HEIGHT']:
    if i>=0 and i<=80:
        Height_new.append(1)
    elif i>=81 and i<=120:
        Height_new.append(2)
    elif i>=121 and i<=150:
        Height_new.append(3)
    elif i>=151 and i<=175:
        Height_new.append(4)
    elif i>=176 and i<=190:
        Height_new.append(5)
    elif i>=191:
        Height_new.append(6)
    elif math.isnan(i):
        Height_new.append(None)

In [13]:
occupants['HEIGHT']=Height_new

C:\Users\Dawid\anaconda3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


KOLUMNA CHTYPE

In [14]:
Chtype_new=[]

for i in occupants['CHTYPE']:
    if i==0:
        Chtype_new.append(0)
    elif i>=1 and i<=8:
        Chtype_new.append(1) 
    elif math.isnan(i):
        Chtype_new.append(None)

In [15]:
occupants['CHTYPE']=Chtype_new

KOLUMNA BAGAVAIL

In [16]:
Bagavail_new=[]

for i in occupants['BAGAVAIL']:
    if i==0 or i==2 or i==3:
        Bagavail_new.append(0) 
    elif i==1:
        Bagavail_new.append(1) 
    elif math.isnan(i):
        Bagavail_new.append(None)

In [17]:
occupants['BAGAVAIL']=Bagavail_new

KOLUMNA AGE:

In [18]:
Age_new=[]

for i in occupants['AGE']:
    if i>=0 and i<=3:
        Age_new.append(1)
    elif i>=4 and i<=7:
        Age_new.append(2)
    elif i>=8 and i<=12:
        Age_new.append(3)
    elif i>=13 and i<=16:
        Age_new.append(4)
    elif i>=17 and i<=21:
        Age_new.append(5)
    elif i>=22 and i<=30:
        Age_new.append(6)
    elif i>=31 and i<=50:
        Age_new.append(7)
    elif i>=51 and i<=65:
        Age_new.append(8)
    elif i>=66:
        Age_new.append(9)
    elif math.isnan(i):
        Age_new.append(None)

In [19]:
occupants['AGE']=Age_new

### Wspolne funkcje - grupowanie

In [20]:
def classify_value(input_value,mappings): 
    
    for mark,values in  mappings.items():
        if input_value in values: 
            return mark 
         
    return None

### Kolumna bodytype

In [21]:
bodytype_mappings = {0:[1,2,3,4,5,6,7,8,9,17],                                       # osobowy
                          1:[61,62,63,64,69,74,78,79],                               # ciezarowka 
                          2:[24,25,28,50,58,59,60],                                  # busy   
                          3:[10,40,41,45,48],                                        # dostawcze 
                          4:[65,68,67,70,80,81,82,89,90],                            # motory&quady
                          5:[12, 14, 15, 16, 19, 20, 21, 22, 29, 30, 31, 32, 33,39], # większe samochody osobowe
                          6:[92, 93, 97, 42, 23, 11]}                                # pozostale 
    

vehicles["BODYTYPE"] = vehicles["BODYTYPE"].apply(classify_value,mappings=bodytype_mappings)


### Kolumna STRATIF 

In [22]:
stratif_mappings =       {0:['A','B'], # smiertelne obrażenia
                          1:['J','K'], # poważne obrażenia – hospitalizowany (min. 1noc)
                          2:['C','D'], #  poważne obrażenia – nie hospitalizowany
                          3:['E','F'], # niewielkie obrażenia, ale przewieziony do szpitala
                          4:['G','H']} # niewielkie obrażenia


accidents["STRATIF"] = accidents["STRATIF"].apply(classify_value,mappings=stratif_mappings)


In [23]:
SEX_new=[]

for i in occupants['SEX']:
    if i==1:
        SEX_new.append(1)
    elif i==2:
        SEX_new.append(2)
    elif i>=3:
        SEX_new.append(3)
    elif math.isnan(i):
        SEX_new.append(None)

In [24]:
occupants['SEX'] = SEX_new

In [25]:
WEIGHT_new=[]

for i in occupants['WEIGHT']:
    if i>=0 and i<=20:
        WEIGHT_new.append(1)
    elif i>=21 and i<=40:
        WEIGHT_new.append(2)
    elif i>=41 and i<=60:
        WEIGHT_new.append(3)
    elif i>=61 and i<=80:
        WEIGHT_new.append(4)
    elif i>=81 and i<=100:
        WEIGHT_new.append(5)
    elif i>100:
        WEIGHT_new.append(6)
    elif math.isnan(i):
        WEIGHT_new.append(None)

In [26]:
occupants['WEIGHT'] = WEIGHT_new

In [27]:
SEATPOS_new=[]

for i in occupants['SEATPOS']:
    if i>=11 and i<=15:
        SEATPOS_new.append(1)
    elif i>=21 and i<=25 or i==29:
        SEATPOS_new.append(2)
    elif i>=31 and i<=59:
        SEATPOS_new.append(3)
    elif i==19 or i==97 or i==98:
        SEATPOS_new.append(4)
    elif math.isnan(i):
        SEATPOS_new.append(None)

In [28]:
occupants['SEATPOS'] = SEATPOS_new

In [29]:
ROLE_new=[]
for i in occupants['ROLE']:
    if i==1:
        ROLE_new.append(1)
    elif i==2:
        ROLE_new.append(2)
    elif math.isnan(i):
        ROLE_new.append(None)

In [30]:
occupants['ROLE'] = ROLE_new

In [31]:
MANUSE_new=[]

for i in occupants['MANUSE']:
    if i>=0 and i<=1:
        MANUSE_new.append(1)
    elif i>=2 and i<=18:
        MANUSE_new.append(2)
    elif math.isnan(i):
        MANUSE_new.append(None)

In [32]:
occupants['MANUSE'] = MANUSE_new

### Stworzenie nowych tabel

In [33]:
occupants.to_csv("./occupants.csv", index = False)
accidents.to_csv("./accidents.csv", index = False)
vehicles.to_csv("./vehicles.csv", index = False)